In [11]:
import pandas as pd
from functions import rolling_calculator
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split


In [12]:
svm_df = pd.read_csv("data/cleaned_merged_seasons.csv")

/var/folders/9_/fcykl83s1dncqz69y6vx8b6w0000gn/T/ipykernel_49402/519723715.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  svm_df = pd.read_csv("data/cleaned_merged_seasons.csv")


In [13]:
svm_df.head

#groupby FWD, MID, DEF, GK - then calculate the PPV by taking points that game over value, then take the statistics of each grouping
svm_df['ppv'] = svm_df['total_points'] / svm_df['value']
svm_df = svm_df[svm_df['minutes'] > 0]

ppv_stats = svm_df.groupby('position')['ppv'].agg([
    ('count', 'count'),
    ('mean', 'mean'),
    ('median', 'median'),
    ('std', 'std'),
    ('min', 'min'),
    ('25%', lambda x: x.quantile(0.25)),
    ('75%', lambda x: x.quantile(0.75)),
    ('max', 'max')
]).round(3)

print("PPV Statistics by Position:")
print(ppv_stats)
print()

PPV Statistics by Position:
          count   mean  median    std    min    25%    75%    max
position                                                         
DEF       18720  0.058   0.033  0.062 -0.143  0.021  0.103  0.525
FWD        7668  0.048   0.028  0.048 -0.041  0.018  0.060  0.392
GK         4097  0.075   0.049  0.060 -0.061  0.036  0.113  0.341
GKP          24  0.062   0.055  0.037  0.000  0.042  0.075  0.160
MID       24825  0.050   0.037  0.047 -0.045  0.020  0.057  0.426



# Without dropping zeros
| Position | Count | Mean  | Median | Std   | Min    | 25%  | 75%   | Max   |
|----------|-------|-------|--------|-------|--------|------|-------|-------|
| DEF      | 43385 | 0.025 | 0.0    | 0.050 | -0.143 | 0.0  | 0.023 | 0.525 |
| FWD      | 16523 | 0.022 | 0.0    | 0.041 | -0.041 | 0.0  | 0.025 | 0.392 |
| GK       | 13994 | 0.022 | 0.0    | 0.047 | -0.061 | 0.0  | 0.022 | 0.341 |
| GKP      | 101   | 0.015 | 0.0    | 0.032 | 0.000  | 0.0  | 0.000 | 0.160 |
| MID      | 52073 | 0.024 | 0.0    | 0.041 | -0.062 | 0.0  | 0.036 | 0.426 |

# With dropping zeros
| Position | Count | Mean  | Median | Std   | Min    | 25%   | 75%   | Max   |
|----------|-------|-------|--------|-------|--------|-------|-------|-------|
| DEF      | 18720 | 0.058 | 0.033  | 0.062 | -0.143 | 0.021 | 0.103 | 0.525 |
| FWD      | 7668  | 0.048 | 0.028  | 0.048 | -0.041 | 0.018 | 0.060 | 0.392 |
| GK       | 4097  | 0.075 | 0.049  | 0.060 | -0.061 | 0.036 | 0.113 | 0.341 |
| GKP      | 24    | 0.062 | 0.055  | 0.037 | 0.000  | 0.042 | 0.075 | 0.160 |
| MID      | 24825 | 0.050 | 0.037  | 0.047 | -0.045 | 0.020 | 0.057 | 0.426 |

Going to attempt to label the games for fwd's that are more than one standard deviation of PPV so (PPV > .096) as "budget" which will give roughly 16% of them.

In [ ]:
#Only take forwards first
svm_df = svm_df[svm_df['position'] == 'FWD']

#Apply mask to calc high_budget
svm_df["high_budget"] = (svm_df['ppv'] > 0.096).astype(int)

to_drop_all_fwd = ['team_a_score','team_h_score', 'was_home', "round",
            "opponent_team", "kickoff_time", "fixture", "position","element", "creativity", "influence", "threat", "bonus"
            , "transfers_in", "transfers_out", "clean_sheets", "penalties_saved", "goals_conceded", "penalties_saved", "saves",
             "season_x", "team_x" , "opp_team_name", "red_cards", "penalties_missed", "yellow_cards"]

svm_df = svm_df.drop(columns=to_drop_all_fwd)

Y = svm_df['high_budget']
svm_df, stat_cols = rolling_calculator(svm_df, 5)
stat_cols.append("name")
svm_df = svm_df.drop(columns=stat_cols)

X = svm_df.sort_index()






KeyError: 'position'

In [18]:
X.sort_index()

,GW,assists_last_5_games,bps_last_5_games,goals_scored_last_5_games,ict_index_last_5_games,minutes_last_5_games,own_goals_last_5_games,selected_last_5_games,total_points_last_5_games,transfers_balance_last_5_games,value_last_5_games,ppv_last_5_games,high_budget_last_5_games
14,1,0.0,-3.0,0.0,4.3,90.0,0.0,410883.0,2.0,0.0,65.0,0.030769,0.0
17,1,0.0,7.0,0.0,3.2,90.0,0.0,268504.0,2.0,0.0,65.0,0.030769,0.0
19,1,2.0,35.0,0.0,10.4,84.0,0.0,213574.0,11.0,0.0,95.0,0.115789,1.0
30,1,0.0,-1.0,0.0,1.5,66.0,0.0,100715.0,2.0,0.0,65.0,0.030769,0.0
33,1,0.0,2.0,0.0,1.0,8.0,0.0,60547.0,1.0,0.0,65.0,0.015385,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
126001,38,0.0,39.0,1.0,15.1,146.0,0.0,518558.0,11.0,-2185.0,361.0,0.152588,1.0
126017,38,0.0,39.0,1.0,8.9,258.0,0.0,1003624.0,13.0,-3575.0,266.0,0.254403,1.0
126021,38,0.0,42.0,1.0,29.0,433.0,0.0,224546.0,14.0,2718.0,235.0,0.297872,1.0
126044,38,1.0,77.0,2.0,28.8,391.0,0.0,4856856.0,21.0,-764610.0,298.0,0.392282,1.0


In [19]:
Y.sort_index()

14        0
17        0
19        1
30        0
33        0
         ..
126001    0
126017    0
126021    0
126044    0
126045    1
Name: high_budget, Length: 7668, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

# 6. Scale features (CRITICAL for SVM!)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 7. Train SVM
svm = SVC(kernel='rbf', C=1.0, gamma='scale', probability=True, random_state=42)
svm.fit(X_train_scaled, y_train)

# 8. Evaluate
y_pred = svm.predict(X_test_scaled)
y_prob = svm.predict_proba(X_test_scaled)[:, 1]